## Module and data

In [1]:
# Load the module for class initialization functions

using ExpectationMaximization
import ExpectationMaximization: initialize_class_assignments, m_step

# Make sure that this notebook is run from the main directory

endswith(pwd(), "/notebooks") && cd("..")
PROJECT_PATH = pwd()

include("$PROJECT_PATH/src/load_datasets.jl")

dataset = load_rte();

# [questions x one-hot classes]
class_assignments = initialize_class_assignments(FDS(), dataset.crowd_counts);

# [question x annotators x classes]
counts = dataset.crowd_counts;

In [2]:
using LinearAlgebra: normalize

function init_param(counts)
    class_marginals, error_rates = m_step(FDS(), counts, initialize_class_assignments(FDS(), counts))
    return normalize(class_marginals[:], 1)
end

second(x) = x[2]
third(x) = x[3]

"""
element-wise multiplication in order to remove annotators that didn't answer the question
"""
function onehot_choices_to_categorical(x::Array{<:Real, 3})::Matrix{Int}
    third.(argmax(x; dims=3)[:, :, 1]) .* maximum(x; dims=3)[:, :, 1]
end

Q, A, O = size(counts) # questions, answers, options
choices = onehot_choices_to_categorical(counts)
class_assignments = second.(argmax(initialize_class_assignments(FDS(), counts); dims=2)[:, 1])
class_marginals = init_param(counts)
;

In [53]:
using Distributions
using LogExpFunctions
using Turing

@model function em_fds(
    choices::Matrix{Int}, # [Q x A]
    unnormalized_class_marginals::Vector{<:Real} # [O] (probability vector)
)
    # Get dimensions
    Q, A = size(choices) # questions, answers
    O = length(unnormalized_class_marginals) # options

    # eq (4)
    p_Y = Categorical(softmax(unnormalized_class_marginals))
    
    # sample answer sheet
    Ys ~ filldist(p_Y, Q)

    # @show Ys
    
    #TODO: 
    # 1. try using broadcasting for S and T
    # 2. figure out NaN problem
    # 3. get initial version pushed early next week (Sunday)

    # eq (3)

    # S = [for a in 1:A, o in 1:O]

    S = zeros(Int, A, O) # [A x O]
    for a in 1:A, o in 1:O
        # {i | Yᵢ = c ∧ a has answered question i}
        S[a, o] = length([q for q in 1:Q 
                          if Ys[q] == o # Yᵢ = c
                          && choices[q, a] != 0 # a has answered question i
                          ]) 
    end

    T = zeros(Int, A, O, O) # [A x O x O]
    for a in 1:A, o1 in 1:O, o2 in 1:O
        # {i | Yᵢ = c ∧ a has answered cₐ on question i}
        T[a, o1, o2] = length([q for q in 1:Q 
                               if Ys[q] == o1 # Yᵢ = c
                               && choices[q, a] == o2 # a has answered cₐ on question i
                               ])
    end
    
    error_rates = T ./ S

    @show error_rates
    
    p_c_given_Y = [Categorical(error_rates[a, o, :]) for a in 1:A, o in 1:O]
    for q in 1:Q, a in 1:A
        choices[q, a] == 0 && continue
        choices[q, a] ~ p_c_given_Y[a, Ys[q]]
    end
end


em_fds (generic function with 2 methods)

In [54]:
function get_latent(chn; keep_pct=.5, use_every=1)
    # [latent_dim, num_samples]
    chn.value.data[floor(Int, keep_pct * end) : use_every : end, 1:800] |> Matrix{Int} |> transpose
end

function make_obj(latent)
    function obj(unnormalized_class_marginals)
        normalized_class_marginals = normalize(unnormalized_class_marginals, 1)
        return -mean([
            logjoint(em_fds(choices, normalized_class_marginals), (; Ys=Ys))
            for Ys in eachcol(latent)
            ])
    end
end

make_obj (generic function with 1 method)

In [55]:
# using Random
# using ProgressMeter
# using Optim

# Random.seed!(42)

# num_particles = 20
# num_samples = 20
# num_iterations = 10
# sampler = PG(num_particles)

# class_marginals = init_param(counts)
# trace = (chn=[], opt=[])

# @showprogress for i in 1:num_iterations
#     # E-step
#     chn = sample(em_fds(choices, class_marginals), sampler, num_samples; progress=false)
#     push!(trace.chn, chn)
    
#     # M-step
#     opt = optimize(make_obj(get_latent(chn)), class_marginals)
#     push!(trace.opt, opt)
#     class_marginals = opt.minimizer

#     # Report
#     println("$i: $(opt.minimum) | $(chn.logevidence)")
# end

# using Dates
# using JLD
# save("./trace_$(split(string(now()), '.')[1]).jld", "trace", trace)

In [ ]:
trace = load("./trace.jld")["trace"]
;

In [57]:
trace|>typeof|>fieldnames, length(trace.chn), length(trace.opt)

((:chn, :opt), 4, 4)

In [ ]:
class_marginals = trace.opt[end].minimizer
chn = sample(em_fds(choices, class_marginals), sampler, num_samples)

In [62]:
error_rates = [0.5769230769230769 0.7857142857142857; 0.4891304347826087 0.4722222222222222; 0.4166666666666667 0.625; 0.48295454545454547 0.4423076923076923; 0.4166666666666667 0.625; 0.5990888382687927 0.6551724137931034; 0.5396825396825397 0.5675675675675675; 0.7331288343558282 0.7897196261682243; 0.8363273453093812 0.782608695652174; 0.7794117647058824 0.75; 0.49122807017543857 0.4782608695652174; 0.28 0.26666666666666666; 0.25 0.0; 0.4666666666666667 0.3; 0.41935483870967744 0.48936170212765956; 0.6 0.676923076923077; 0.4375 0.75; 0.8518518518518519 0.8461538461538461; 0.9333333333333333 0.8; 0.4 0.8; 0.4722222222222222 0.5833333333333334; 0.5 0.5; 0.4666666666666667 0.0; 0.42857142857142855 0.3888888888888889; 0.4666666666666667 0.2; 0.4 0.4; 0.6 0.2; 0.5333333333333333 0.2; 0.5 0.3333333333333333; 0.42857142857142855 0.3333333333333333; 0.45454545454545453 0.4444444444444444; 0.5555555555555556 0.6363636363636364; 0.48484848484848486 0.5185185185185185; 0.4444444444444444 0.45454545454545453; 0.35714285714285715 0.5; 0.35714285714285715 0.16666666666666666; 0.35714285714285715 0.25; 0.2857142857142857 0.5; 0.7142857142857143 0.5; 0.6666666666666666 0.45454545454545453; 0.47368421052631576 0.2857142857142857; 0.45161290322580644 0.1724137931034483; 0.8888888888888888 0.45454545454545453; 0.7333333333333333 0.8; 0.6666666666666666 0.375; 0.6666666666666666 0.4; 0.3333333333333333 0.40625; 0.3076923076923077 0.8571428571428571; 0.8461538461538461 1.0; 0.23076923076923078 0.5714285714285714; 0.4230769230769231 0.35714285714285715; 0.6153846153846154 0.8571428571428571; 0.6153846153846154 0.7142857142857143; 0.5217391304347826 0.5294117647058824; 0.5 0.625; 0.5849056603773585 0.6666666666666666; 0.4166666666666667 0.5; 0.3333333333333333 0.75; 0.5 0.625; 0.5 0.625; 0.3 0.5; 0.3 0.3; 0.1 0.4; 0.5 0.4; 0.0 0.2; 0.5625 0.5; 0.5625 0.25; 0.4523809523809524 0.4444444444444444; 0.6875 0.5; 0.375 0.5; 0.5625 0.5; 0.6153846153846154 0.7142857142857143; 0.6153846153846154 0.5714285714285714; 0.38461538461538464 0.5714285714285714; 0.46153846153846156 0.5714285714285714; 0.42857142857142855 0.3333333333333333; 0.5714285714285714 0.8333333333333334; 0.5 0.6666666666666666; 0.6428571428571429 0.8333333333333334; 0.7857142857142857 0.6666666666666666; 0.6363636363636364 0.6666666666666666; 0.18181818181818182 0.3333333333333333; 0.5909090909090909 0.5555555555555556; 0.6363636363636364 0.6666666666666666; 0.3 0.3; 0.625 0.375; 0.5 0.6; 1.0 1.0; 0.7692307692307693 0.42857142857142855; 0.6153846153846154 0.42857142857142855; 0.7692307692307693 0.5714285714285714; 0.5 0.16666666666666666; 0.5714285714285714 0.3333333333333333; 0.5714285714285714 0.16666666666666666; 0.2727272727272727 0.5555555555555556; 0.18181818181818182 0.6666666666666666; 0.2727272727272727 0.5555555555555556; 0.18181818181818182 0.6666666666666666; 0.75 0.5; 0.5 0.3; 0.4358974358974359 0.2857142857142857; 0.42857142857142855 0.16666666666666666; 0.14285714285714285 0.0; 0.35714285714285715 0.0; 0.2857142857142857 0.16666666666666666; 0.42857142857142855 0.16666666666666666; 0.2857142857142857 0.15789473684210525; 0.4166666666666667 0.25; 0.3333333333333333 0.25; 0.1111111111111111 0.09090909090909091; 0.1111111111111111 0.09090909090909091; 0.3333333333333333 0.18181818181818182; 0.3333333333333333 0.5; 0.5 0.625; 0.5 0.625; 0.5 0.625; 0.4166666666666667 0.625; 0.4 0.6; 0.38461538461538464 0.42857142857142855; 0.4 0.6; 0.7272727272727273 0.6666666666666666; 0.5769230769230769 0.7142857142857143; 0.45454545454545453 0.5555555555555556; 0.5757575757575758 0.6296296296296297; 0.5416666666666666 0.4375; 0.4 0.6; 0.4 0.4; 0.48 0.7333333333333333; 0.5 0.2; 0.4 0.2; 0.3 0.1; 0.3 0.1; 0.6428571428571429 0.5; 0.42857142857142855 0.3333333333333333; 0.38461538461538464 0.5714285714285714; 0.5384615384615384 0.5714285714285714; 0.38461538461538464 0.42857142857142855; 0.38461538461538464 0.5714285714285714; 0.5714285714285714 0.5; 0.5 0.16666666666666666; 0.5714285714285714 0.3333333333333333; 0.4666666666666667 0.4; 0.3333333333333333 0.6; 0.5333333333333333 0.4; 0.391304347826087 0.5882352941176471; 0.5 0.6; 0.7 0.7; 0.8 0.4; 0.5 0.6; 0.3076923076923077 0.42857142857142855; 0.07692307692307693 0.2857142857142857; 0.5384615384615384 0.5714285714285714; 0.625 0.6666666666666666; 0.5 0.5833333333333334; 0.5 0.5; 0.875 0.8333333333333334; 0.5 0.5833333333333334; 0.36363636363636365 0.3333333333333333; 0.2727272727272727 0.5555555555555556; 0.45454545454545453 0.3333333333333333; 0.6363636363636364 0.1111111111111111; 0.6666666666666666 0.3125; 0.6923076923076923 0.2857142857142857; 0.6153846153846154 0.2857142857142857;;; 0.4230769230769231 0.21428571428571427; 0.5108695652173914 0.5277777777777778; 0.5833333333333334 0.375; 0.5170454545454546 0.5576923076923077; 0.5833333333333334 0.375; 0.4009111617312073 0.3448275862068966; 0.4603174603174603 0.43243243243243246; 0.2668711656441718 0.2102803738317757; 0.16367265469061876 0.21739130434782608; 0.22058823529411764 0.25; 0.5087719298245614 0.5217391304347826; 0.72 0.7333333333333333; 0.75 1.0; 0.5333333333333333 0.7; 0.5806451612903226 0.5106382978723404; 0.4 0.3230769230769231; 0.5625 0.25; 0.14814814814814814 0.15384615384615385; 0.06666666666666667 0.2; 0.6 0.2; 0.5277777777777778 0.4166666666666667; 0.5 0.5; 0.5333333333333333 1.0; 0.5714285714285714 0.6111111111111112; 0.5333333333333333 0.8; 0.6 0.6; 0.4 0.8; 0.4666666666666667 0.8; 0.5 0.6666666666666666; 0.5714285714285714 0.6666666666666666; 0.5454545454545454 0.5555555555555556; 0.4444444444444444 0.36363636363636365; 0.5151515151515151 0.48148148148148145; 0.5555555555555556 0.5454545454545454; 0.6428571428571429 0.5; 0.6428571428571429 0.8333333333333334; 0.6428571428571429 0.75; 0.7142857142857143 0.5; 0.2857142857142857 0.5; 0.3333333333333333 0.5454545454545454; 0.5263157894736842 0.7142857142857143; 0.5483870967741935 0.8275862068965517; 0.1111111111111111 0.5454545454545454; 0.26666666666666666 0.2; 0.3333333333333333 0.625; 0.3333333333333333 0.6; 0.6666666666666666 0.59375; 0.6923076923076923 0.14285714285714285; 0.15384615384615385 0.0; 0.7692307692307693 0.42857142857142855; 0.5769230769230769 0.6428571428571429; 0.38461538461538464 0.14285714285714285; 0.38461538461538464 0.2857142857142857; 0.4782608695652174 0.47058823529411764; 0.5 0.375; 0.41509433962264153 0.3333333333333333; 0.5833333333333334 0.5; 0.6666666666666666 0.25; 0.5 0.375; 0.5 0.375; 0.7 0.5; 0.7 0.7; 0.9 0.6; 0.5 0.6; 1.0 0.8; 0.4375 0.5; 0.4375 0.75; 0.5476190476190477 0.5555555555555556; 0.3125 0.5; 0.625 0.5; 0.4375 0.5; 0.38461538461538464 0.2857142857142857; 0.38461538461538464 0.42857142857142855; 0.6153846153846154 0.42857142857142855; 0.5384615384615384 0.42857142857142855; 0.5714285714285714 0.6666666666666666; 0.42857142857142855 0.16666666666666666; 0.5 0.3333333333333333; 0.35714285714285715 0.16666666666666666; 0.21428571428571427 0.3333333333333333; 0.36363636363636365 0.3333333333333333; 0.8181818181818182 0.6666666666666666; 0.4090909090909091 0.4444444444444444; 0.36363636363636365 0.3333333333333333; 0.7 0.7; 0.375 0.625; 0.5 0.4; 0.0 0.0; 0.23076923076923078 0.5714285714285714; 0.38461538461538464 0.5714285714285714; 0.23076923076923078 0.42857142857142855; 0.5 0.8333333333333334; 0.42857142857142855 0.6666666666666666; 0.42857142857142855 0.8333333333333334; 0.7272727272727273 0.4444444444444444; 0.8181818181818182 0.3333333333333333; 0.7272727272727273 0.4444444444444444; 0.8181818181818182 0.3333333333333333; 0.25 0.5; 0.5 0.7; 0.5641025641025641 0.7142857142857143; 0.5714285714285714 0.8333333333333334; 0.8571428571428571 1.0; 0.6428571428571429 1.0; 0.7142857142857143 0.8333333333333334; 0.5714285714285714 0.8333333333333334; 0.7142857142857143 0.8421052631578947; 0.5833333333333334 0.75; 0.6666666666666666 0.75; 0.8888888888888888 0.9090909090909091; 0.8888888888888888 0.9090909090909091; 0.6666666666666666 0.8181818181818182; 0.6666666666666666 0.5; 0.5 0.375; 0.5 0.375; 0.5 0.375; 0.5833333333333334 0.375; 0.6 0.4; 0.6153846153846154 0.5714285714285714; 0.6 0.4; 0.2727272727272727 0.3333333333333333; 0.4230769230769231 0.2857142857142857; 0.5454545454545454 0.4444444444444444; 0.42424242424242425 0.37037037037037035; 0.4583333333333333 0.5625; 0.6 0.4; 0.6 0.6; 0.52 0.26666666666666666; 0.5 0.8; 0.6 0.8; 0.7 0.9; 0.7 0.9; 0.35714285714285715 0.5; 0.5714285714285714 0.6666666666666666; 0.6153846153846154 0.42857142857142855; 0.46153846153846156 0.42857142857142855; 0.6153846153846154 0.5714285714285714; 0.6153846153846154 0.42857142857142855; 0.42857142857142855 0.5; 0.5 0.8333333333333334; 0.42857142857142855 0.6666666666666666; 0.5333333333333333 0.6; 0.6666666666666666 0.4; 0.4666666666666667 0.6; 0.6086956521739131 0.4117647058823529; 0.5 0.4; 0.3 0.3; 0.2 0.6; 0.5 0.4; 0.6923076923076923 0.5714285714285714; 0.9230769230769231 0.7142857142857143; 0.46153846153846156 0.42857142857142855; 0.375 0.3333333333333333; 0.5 0.4166666666666667; 0.5 0.5; 0.125 0.16666666666666666; 0.5 0.4166666666666667; 0.6363636363636364 0.6666666666666666; 0.7272727272727273 0.4444444444444444; 0.5454545454545454 0.6666666666666666; 0.36363636363636365 0.8888888888888888; 0.3333333333333333 0.6875; 0.3076923076923077 0.7142857142857143; 0.38461538461538464 0.7142857142857143]
p_c_given_Y = [Categorical(error_rates[a, o, :]) for a in 1:A, o in 1:O]
;

In [63]:
Ys = [1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2]
;
length(Ys), sum(Ys)

(800, 1072)

In [68]:
S = zeros(Int, A, O) # [A x O]
for a in 1:A, o in 1:O
    # {i | Yᵢ = c ∧ a has answered question i}
    S[a, o] = length([q for q in 1:Q 
                        if Ys[q] == o # Yᵢ = c
                        && choices[q, a] != 0 # a has answered question i
                        ]) 
end
S_ = 1
S == S_

false

In [74]:
size(S)

(164, 2)

In [38]:
T = zeros(Int, A, O, O) # [A x O x O]
for a in 1:A, o1 in 1:O, o2 in 1:O
    # {i | Yᵢ = c ∧ a has answered cₐ on question i}
    T[a, o1, o2] = length([q for q in 1:Q 
                            if Ys[q] == o1 # Yᵢ = c
                            && choices[q, a] == o2 # a has answered cₐ on question i
                            ])
end
size(T)

(164, 2, 2)

In [60]:
error_rates  = T ./ S
p_c_given_Y = [Categorical(error_rates[a, o, :]) for a in 1:A, o in 1:O]
;


In [28]:
Q, A = size(choices)
O = length(class_marginals);
println("Q = $Q\nA = $A\nO = $O")

Q = 800
A = 164
O = 2


In [44]:
class_marginals = trace.opt[end].minimizer
p_Y = Categorical(softmax(class_marginals))
Ys ~ filldist(p_Y, Q)

LoadError: MethodError: no method matching ~(::Product{Discrete, Categorical{Float64, Vector{Float64}}, FillArrays.Fill{Categorical{Float64, Vector{Float64}}, 1, Tuple{Base.OneTo{Int64}}}}, ::Product{Discrete, Categorical{Float64, Vector{Float64}}, FillArrays.Fill{Categorical{Float64, Vector{Float64}}, 1, Tuple{Base.OneTo{Int64}}}})

2022-09-18T10:00:04.032

In [47]:
@benchmark model.f(
    model,
    Turing.VarInfo(model),
    Turing.SamplingContext(
        Random.GLOBAL_RNG, Turing.SampleFromPrior(), Turing.DefaultContext(),
    ),
    model.args...
)

BenchmarkTools.Trial: 54 samples with 1 evaluation.
 Range (min … max):  87.521 ms … 100.229 ms  ┊ GC (min … max): 0.00% … 7.39%
 Time  (median):     92.469 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   94.016 ms ±   4.175 ms  ┊ GC (mean ± σ):  3.75% ± 3.90%

     ▁  ▁ ▁  ▄ ▁▁       ▄                        █▄ ▁  ▁  ▁     
  ▆▁▁█▆▆█▁█▆▁█▆██▆▁▆▆▆▆▆█▁▆▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▁▆▆▆▁▆▁██▆█▆▆█▆▆█▆▁▆ ▁
  87.5 ms         Histogram: frequency by time          100 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 1297043.